# Library

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm

# Data Load

In [2]:
data_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_comments_cleaned_final.csv"
df = pd.read_csv(data_path)

In [3]:
df.info

<bound method DataFrame.info of            nickname        date  \
0            B Choi  2020-01-01   
1               NaN  2020-01-01   
2        Joe shrimp  2020-01-01   
3       Pagan Ethos  2020-01-01   
4       woojoon Jin  2020-01-01   
...             ...         ...   
283710        제갈 공명  2024-12-31   
283711     Re START  2024-12-31   
283712      yes you  2024-12-31   
283713         주수 김  2024-12-31   
283714  geonseol le  2024-12-31   

                                                  content  
0       유로와 캐나다 달러 쪽이 심상찮음. 달러 폭락을 기정사실화 하는 흐름 확인 필요. ...  
1                             원화 가치 상승은 기정사실 1100원대가 눈앞이다  
2       볼커룰 걔정안은 은행권의 적응 기간을 고려해 2020년 내년 1월 1일부터 발효 아...  
3       금리인하는 미국보다 한국이 선빵을 날릴 것으로 예상합니다. 미국은 내년 상반기까지는...  
4       떡국 한그릇들 하셨나요. 달러원이 결국 본질가치로 회귀해서 1150원대까지 왔네요....  
...                                                   ...  
283710  정부, 내년도에 1년물 단기 원화표시 외평채 20조원 발행 계획 발표. 대기업 법인...  
283711                         내는 애국자답게 서학과 외인이럴 응원혀버리긋다.  
283712 

In [9]:
# 모델용 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBERT-SC")

# 토큰 길이 계산
df["token_length"] = df["content"].apply(lambda x: len(tokenizer.tokenize(str(x))))

# 길이 128 미만인 댓글만 필터링
df_filtered = df[df["token_length"] < 128].copy()

# token_length 컬럼은 삭제해도 됨
df_filtered.drop(columns=["token_length"], inplace=True)

df_filtered.info

<bound method DataFrame.info of            nickname        date  \
0            B Choi  2020-01-01   
1               NaN  2020-01-01   
2        Joe shrimp  2020-01-01   
4       woojoon Jin  2020-01-01   
5               a j  2020-01-02   
...             ...         ...   
283710        제갈 공명  2024-12-31   
283711     Re START  2024-12-31   
283712      yes you  2024-12-31   
283713         주수 김  2024-12-31   
283714  geonseol le  2024-12-31   

                                                  content  
0       유로와 캐나다 달러 쪽이 심상찮음. 달러 폭락을 기정사실화 하는 흐름 확인 필요. ...  
1                             원화 가치 상승은 기정사실 1100원대가 눈앞이다  
2       볼커룰 걔정안은 은행권의 적응 기간을 고려해 2020년 내년 1월 1일부터 발효 아...  
4       떡국 한그릇들 하셨나요. 달러원이 결국 본질가치로 회귀해서 1150원대까지 왔네요....  
5                                      달러 하락 1100원 이탈시 매수  
...                                                   ...  
283710  정부, 내년도에 1년물 단기 원화표시 외평채 20조원 발행 계획 발표. 대기업 법인...  
283711                         내는 애국자답게 서학과 외인이럴 응원혀버리긋다.  
283712 

# Analyze Sentiment with Fine Tuning Model

In [4]:
# 파인 튜닝한 모델 불러오기
model_name = "DataWizardd/finbert-sentiment-krw-comment-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/455k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/406M [00:00<?, ?B/s]

In [5]:
sentiment_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [6]:
def analyze_sentiment(text):
    if pd.isna(text) or not text.strip():
        return ("None", 0.0)
    result = sentiment_pipe(text, truncation=True)[0]
    return result["label"], result["score"]

In [10]:
tqdm.pandas()
df_filtered[["sentiment", "score"]] = df_filtered["content"].progress_apply(lambda x: pd.Series(analyze_sentiment(x)))

100%|█████████████████████████████████████████████████████████████████████████| 280843/280843 [30:11<00:00, 155.06it/s]


In [11]:
# 긍정/부정 개수 카운트
print(df_filtered["sentiment"].value_counts())

sentiment
LABEL_1    191193
LABEL_0     89252
None          398
Name: count, dtype: int64


In [12]:
# "None"이 문자열로 들어간 경우 필터링
df_filtered = df_filtered[df_filtered["sentiment"] != "None"].copy()

In [13]:
print(df_filtered["sentiment"].value_counts())

sentiment
LABEL_1    191193
LABEL_0     89252
Name: count, dtype: int64


In [14]:
save_path = r"C:\Users\james\J_Data_Lab\Project-FXCast\crawler\data\investing_content_sentiment.csv"
df_filtered.to_csv(save_path, index=False, encoding="utf-8-sig")